In [1]:
import numpy as np
import pandas as pd
import csv
from pandas import Series,DataFrame

In [2]:
def dateprocessing(data_train,data_test):
    y = data_train['Survived'] # 将训练集Survived 数据存储在y中
    del data_train['Survived'] # 删除训练集Survived数据
    sum_id = data_test['PassengerId'] # 存储测试集乘客ID
    df = pd.merge(data_train, data_test,how='outer') # 合并无Survived数据的训练集与测验集，how = ‘outer’ 意为并集
    df = df.drop(['Name','Ticket','Cabin','Age','Fare','SibSp','Parch','Embarked','PassengerId'],axis=1)# #删掉无关因素，axis=0 删除行，=1 删除列
    dumm = pd.get_dummies(df['Sex']) # '哑变量'矩阵，就是把Sex选项放到列名来，Sex_male和Sex_female，然后他对应的是0是否，1是是
    dumm1 = pd.get_dummies(df['Pclass']) # '哑变量'矩阵，与Sex同理
    df = df.join(dumm)# 把哑变量矩阵添加上去
    df = df.join(dumm1)
    del df['Sex'] #删除Sex列
    del df['Pclass']#删除Pclass列
    data_train = df[:len(data_train)]#第len(data_train)行之前的数据（不含第len(data_train)行）
    data_test = df[len(data_train):]#第len(data_train)行及之后的行，注意冒号是必须的
    data_train=data_train.join(y)
    return data_train,data_test

In [3]:
def NaiveBayes_ParameterCalculate(data_train):
    #Sex与Survived的分组
    df = data_train.drop(['Name','Ticket','Cabin','Age','Fare','SibSp','Parch','Embarked'],axis=1) # 删除姓名、ID、船票信息、客舱信息，axis=0 删除行，=1 删除列
    g = df.groupby(['Sex','Survived']) # 数据分数，里面的那个groupby就是分组命令
    df1 = pd.DataFrame(g.count()['PassengerId'])# 引号里面只是那列数值的标题
    #Pclass与Survived的分组
    g = df.groupby(['Pclass','Survived']) # 数据分数，里面的那个groupby就是分组命令
    df2 = pd.DataFrame(g.count()['PassengerId'])# 引号里面只是那列数值的标题
    
    P_survived=(df1.iloc[1]+df1.iloc[3])/891#计算生存的概率
    P_unsurvived=(df1.iloc[0]+df1.iloc[2])/891#计算死亡的概率
    P_survived_male=(df1.iloc[3])/(df1.iloc[1]+df1.iloc[3])#计算男性生存的概率
    P_survived_female=(df1.iloc[1])/(df1.iloc[1]+df1.iloc[3])#计算女性生存的概率
    P_unsurvived_male=(df1.iloc[2])/(df1.iloc[0]+df1.iloc[2])#计算男性死亡的概率
    P_unsurvived_female=(df1.iloc[0])/(df1.iloc[0]+df1.iloc[2])#计算女性生存的概率
    P_survived_Pclass1=(df2.iloc[1])/(df2.iloc[1]+df2.iloc[3]+df2.iloc[5])#计算Pclass1生存的概率
    P_unsurvived_Pclass1=(df2.iloc[0])/(df2.iloc[0]+df2.iloc[2]+df2.iloc[4])#计算Pclass1死亡的概率
    P_survived_Pclass2=(df2.iloc[3])/(df2.iloc[1]+df2.iloc[3]+df2.iloc[5])#计算Pclass2生存的概率
    P_unsurvived_Pclass2=(df2.iloc[2])/(df2.iloc[0]+df2.iloc[2]+df2.iloc[4])#计算Pclass2死亡的概率
    P_survived_Pclass3=(df2.iloc[5])/(df2.iloc[1]+df2.iloc[3]+df2.iloc[5])#计算Pclass3生存的概率
    P_unsurvived_Pclass3=(df2.iloc[4])/(df2.iloc[0]+df2.iloc[2]+df2.iloc[4])#计算Pclass3死亡的概率
    parameters = {"P_survived": P_survived,
                  "P_unsurvived": P_unsurvived,
                  "P_survived_male": P_survived_male,
                  "P_survived_female": P_survived_female,
                  "P_unsurvived_male": P_unsurvived_male,
                  "P_unsurvived_female": P_unsurvived_female,
                  "P_survived_Pclass1": P_survived_Pclass1,
                  "P_unsurvived_Pclass1": P_unsurvived_Pclass1,
                  "P_survived_Pclass2": P_survived_Pclass2,
                  "P_unsurvived_Pclass2": P_unsurvived_Pclass2,
                  "P_survived_Pclass3": P_survived_Pclass3,
                  "P_unsurvived_Pclass3": P_unsurvived_Pclass3,}
    return parameters  

In [4]:
def NaiveBayes_Classifier(data_train,data_test,parameters):
    data_train=np.array(data_train)
    data_test=np.array(data_test)
    P_likelihood_survived=np.array([parameters["P_survived_female"],parameters["P_survived_male"],parameters["P_survived_Pclass1"],parameters["P_survived_Pclass2"],parameters["P_survived_Pclass3"]])
    P_likelihood_unsurvived=np.array([parameters["P_unsurvived_female"],parameters["P_unsurvived_male"],parameters["P_unsurvived_Pclass1"],parameters["P_unsurvived_Pclass2"],parameters["P_unsurvived_Pclass3"]])
    #训练集分类
    #计算生存下来的概率
    a=np.multiply(data_train, P_likelihood_survived.T)
    a=np.sort(a,axis = 1)
    b=a.T[3:4]
    c=a.T[4:]
    d=np.multiply(b,c)
    e=np.array(parameters["P_survived"])
    P_Trainforecast_survived=d*e
    #计算未生存下来的概率
    a=np.multiply(data_train, P_likelihood_unsurvived.T)
    a=np.sort(a,axis = 1)
    b=a.T[3:4]
    c=a.T[4:]
    d=np.multiply(b,c)
    e=np.array(parameters["P_unsurvived"])
    P_Trainforecast_unsurvived=d*e
    #两个概率相减，大于0的为生存，赋值为1，小于0的死亡，赋值为0
    P_Trainforecast_Result=P_Trainforecast_survived-P_Trainforecast_unsurvived
    P_Trainforecast_Result=np.int64(P_Trainforecast_Result>0)
    P_Trainforecast_Result=P_Trainforecast_Result.T
    #测试集分类
    #计算生存下来的概率
    a=np.multiply(data_test, P_likelihood_survived.T)
    a=np.sort(a,axis = 1)
    b=a.T[3:4]
    c=a.T[4:]
    d=np.multiply(b,c)
    e=np.array(parameters["P_survived"])
    P_Testforecast_survived=d*e
    #计算未生存下来的概率
    a=np.multiply(data_test, P_likelihood_unsurvived.T)
    a=np.sort(a,axis = 1)
    b=a.T[3:4]
    c=a.T[4:]
    d=np.multiply(b,c)
    e=np.array(parameters["P_unsurvived"])
    P_Testforecast_unsurvived=d*e
    #两个概率相减，大于0的为生存，赋值为1，小于0的死亡，赋值为0
    P_Testforecast_Result=P_Testforecast_survived-P_Testforecast_unsurvived
    P_Testforecast_Result=np.int64(P_Testforecast_Result>0)
    P_Testforecast_Result=P_Testforecast_Result.T
    return P_Trainforecast_Result,P_Testforecast_Result

    
    

In [10]:
def Titanic_NaiveBayes_Model():
    Original_data_test = pd.read_csv(r'test.csv') # 导入测验集数据
    Original_data_train = pd.read_csv(r'train.csv') #根据数据位置自行修改
    data_train,data_test=dateprocessing(Original_data_train,Original_data_test)
    Original_data_train = pd.read_csv(r'train.csv') #根据数据位置自行修改 
    parameters=NaiveBayes_ParameterCalculate(Original_data_train)
    y_train = data_train['Survived'] # 将训练集Survived 数据存储在y中
    del data_train['Survived'] # 删除训练集Survived数据
    y_train=np.array(y_train)
    l_train=len(y_train)
    y_train=y_train.reshape(l_train,1)
    Original_data_test = pd.read_csv(r'test.csv') # 导入测验集数据
    Original_data_train = pd.read_csv(r'train.csv') #根据数据位置自行修改
    P_Trainforecast_Result,P_Testforecast_Result=NaiveBayes_Classifier(data_train,data_test,parameters)
    #计算训练组的正确率
    y_train=np.abs(y_train-P_Trainforecast_Result)
    error_number=np.sum(y_train,axis=0)
    P_Trainforecast_accuracy=(l_train-error_number)/l_train
    #计算训练组的正确率
    Test_Result=pd.read_csv(r'gender_submission.csv')
    y_test = Test_Result['Survived'] # 将训练集Survived 数据存储在y中
    y_test=np.array(y_test)
    l_test=len(y_test)
    y_test=y_test.reshape(l_test,1)
    y_test=np.abs(y_test-P_Testforecast_Result)
    error_number=np.sum(y_test,axis=0)
    P_Testforecast_accuracy=(l_test-error_number)/l_test
    return P_Trainforecast_accuracy,P_Testforecast_accuracy,P_Testforecast_Result


In [11]:
P_Trainforecast_accuracy,P_Testforecast_accuracy,P_Testforecast_Result=Titanic_NaiveBayes_Model()
print(P_Trainforecast_accuracy)
print(P_Testforecast_accuracy)

[0.78675645]
[1.]


In [13]:
Test_Result=pd.read_csv(r'gender_submission.csv')
y_test = Test_Result['Survived'] # 将训练集Survived 数据存储在y中
y_test=np.array(y_test)
print(y_test)#这个是原来的数据集
print(P_Testforecast_Result.T)#这个是预测的数据集

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]
[[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0
  1 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0
  1